### 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab12_2 RNN (toy example)
hihello라는 sequence에서 'hihell' sequence를 입력으로 받으면, 'ihello' sequence가 나오도록 RNN을 이용해서 training 합니다.

In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777) # for reproducibility

### pre-processing 

In [2]:
# Teach hello: hihell -> ihello
idx2char = ['h', 'i', 'e', 'l', 'o']
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
y_data = [[1, 0, 2, 3, 3, 4]] #ihello
# h 0, i 1, e 2 ,l 3, o 4

x_one_hot = []
#y_one_hot = []

for i in x_data[0]:
    tmp = list(np.repeat(0,len(idx2char)))
    tmp[i] = 1
    x_one_hot.append(tmp)
x_one_hot = [x_one_hot]
x_one_hot
#for i in y_data[0]:
#    tmp = list(np.repeat(0,len(idx2char)))
#    tmp[i] = 1
#    y_one_hot.append(tmp)
#y_one_hot = [y_one_hot]    
#print(x_one_hot)
#print(y_one_hot)

[[[1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [1, 0, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 1, 0]]]

### Generate many-to-many RNN

In [3]:
# hyper-parameter setting
num_classes = len(idx2char)
input_dim = len(idx2char) # one-hot size
hidden_size = len(idx2char) # output from the LSTM, 5 to directly predict one-hot
batch_size = len(x_data) # one sentence
sequence_length = len(x_data[0])

In [4]:
# graph setting
X = tf.placeholder(dtype = tf.float32, shape = [None, sequence_length, input_dim]) # batch, sequence, dim
Y = tf.placeholder(dtype = tf.int32, shape = [None, sequence_length]) # batch, sequence 
# (배치단위로 sequence 만큼 one-hot encoding이 아닌 True label이 들어있다.)

# Declare LSTM cell
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, forget_bias = 0.9, state_is_tuple = True)
initial_state = cell.zero_state(batch_size = batch_size, dtype = tf.float32)
outputs, states = tf.nn.dynamic_rnn(cell = cell, inputs = X, sequence_length = [sequence_length],
                                    initial_state = initial_state, dtype = tf.float32)

# Declare Dense layer
outputs_for_fc = tf.reshape(tensor = outputs, shape = [-1, hidden_size])
fc_w = tf.get_variable(name = 'fc_w', shape = [hidden_size, num_classes], dtype = tf.float32, initializer = tf.contrib.layers.xavier_initializer())
fc_b = tf.Variable(initial_value = tf.random_normal(shape = [num_classes], mean = 0.0, stddev = 1.0, dtype = tf.float32))
score = tf.matmul(outputs_for_fc, fc_w) + fc_b

# Calculate seq2seq loss
score_for_loss = tf.reshape(tensor = score, shape = [batch_size, sequence_length, num_classes])
sequence_weights = tf.ones(shape = [batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = score_for_loss, targets = Y, weights = sequence_weights)
loss = tf.reduce_mean(sequence_loss)

# Declare optimizer
train = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(loss)

# Prediction
prediction = tf.argmax(score_for_loss, axis = 2)

In [5]:
# Training 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(1000):
        loss_val, _ = sess.run([loss, train], feed_dict = {X : x_one_hot, Y : y_data})
        hat = sess.run(prediction, feed_dict = {X : x_one_hot})
        if step % 100 == 0:
            print('step : {:4}, loss : {:.3f}, prediction : {}, true Y : {}'.format(step, loss_val, hat, y_data))
            result_str = [idx2char[c] for c in np.squeeze(hat)]
            print('result_str : ', ''.join(result_str))

step :    0, loss : 1.622, prediction : [[3 3 3 3 3 3]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  llllll
step :  100, loss : 0.237, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  200, loss : 0.039, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  300, loss : 0.016, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  400, loss : 0.009, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  500, loss : 0.006, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  600, loss : 0.004, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  700, loss : 0.003, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  800, loss : 0.003, prediction : [[1 0 2 3 3 4]], true Y : [[1, 0, 2, 3, 3, 4]]
result_str :  ihello
step :  900, loss : 0.002, p